# Data base queury to identify transcription start sites.

© 2022 Tom Röschinger. This work is licensed under a <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution License CC-BY 4.0</a>. All code contained herein is licensed under an <a href="https://opensource.org/licenses/MIT">MIT license</a>

***

In this notebook we show how we look for transcription start sites in various data bases. We downloaded [Ecocyc](https://ecocyc.org/) and [Regulon DB](https://regulondb.ccg.unam.mx/). The goal is to get a list that gives us transcription start sites for all genes, such that we can design oligos for a Reg-Seq experiment.

In [59]:
using CSV, DataFrames, wgregseq

┌ Info: Precompiling wgregseq [57506705-3e72-4664-96ef-b7f153ccca47]
└ @ Base loading.jl:1423


## Ecocyc

### Genes

The files come in `.dat` type, so we have some work to do in order to extract the correct information. First we need to read in the file.

In [9]:
s = open("../data/ecocyc/genes.dat") do file
    read(file, String)
end

"# Copyright SRI International 1999-2021, Marine Biological Laboratory 1998-2001, DoubleTwist Inc 1998-1999.  All Rights Reserved.\n#\n# Authors:\n#    Peter D. Karp\n#    Ingrid Keseler\n#    Carol Fulcher\n#    Anamika Kothari\n#    Suzanne Paley\n#    Markus Krummenacker\n#  " ⋯ 4641505 bytes ⋯ "NS - OBS0-439\nKNOCKOUT-GROWTH-OBSERVATIONS - OBS0-49\nKNOCKOUT-GROWTH-OBSERVATIONS - OBS0-37\nKNOCKOUT-GROWTH-OBSERVATIONS - OBS0-33\nLAST-UPDATE - 3839010085\nLEFT-END-POSITION - 2592762\nPRODUCT - MONOMER0-2685\nRIGHT-END-POSITION - 2592962\nTRANSCRIPTION-DIRECTION - +\n//\n"

The entries in this file are separated by `//`, therefore we can use it to separate the entries. The first two entries are comments, so we skip those. Then, each entry is a gene.

In [16]:
# Split genes and drop comments
gene_list = split(s, "//")[3:end]
println(gene_list[1])


UNIQUE-ID - EG11120
TYPES - BC-8.1
COMMON-NAME - tfaE
ACCESSION-1 - b1156
ACCESSION-2 - ECK1142
CENTISOME-POSITION - 26.036354    
COMPONENT-OF - COLI-K12-24
COMPONENT-OF - TU0-13152
DBLINKS - (REGULONDB "b1156" NIL |kothari| 3713101626 NIL NIL)
DBLINKS - (STRING "511145.b1156" NIL |kothari| 3652459938 NIL NIL)
DBLINKS - (ECOLIHUB "tfaE" NIL |kr| 3474243545 NIL NIL)
DBLINKS - (ASAP "ABE-0003880" NIL |paley| 3398447606 NIL NIL)
DBLINKS - (ECHOBASE "EB1110" NIL |pkarp| 3346767937 NIL NIL)
DBLINKS - (OU-MICROARRAY "b1156" NIL NIL NIL NIL NIL)
IN-GROUP - PARALOGOUS-GENE-GROUP-154
INSTANCE-NAME-TEMPLATE - G-*
KNOCKOUT-GROWTH-OBSERVATIONS - OBS0-439
KNOCKOUT-GROWTH-OBSERVATIONS - OBS0-49
KNOCKOUT-GROWTH-OBSERVATIONS - OBS0-37
KNOCKOUT-GROWTH-OBSERVATIONS - OBS0-33
KNOCKOUT-GROWTH-OBSERVATIONS - OBS0-44
LAST-UPDATE - 3701017116
LEFT-END-POSITION - 1208517
MEMBER-SORT-FN - NUMBERED-CLASS-SORT-FN
PRODUCT - EG11120-MONOMER
RIGHT-END-POSITION - 1209119
SYNONYMS - ycfA
TRANSCRIPTION-DIRECTION - -

For each gene, there are multiple attributes, where each attribute is in a single line and has the structure `<attribute> - <value>`. Hence, we can go through every line, look for the attributes we are interested in, and then extract the information. The information gets transformed into an array of the form `[<attribute>, <value>]` for easier access.

In [17]:
# Separate attributes
gene_list = [split(x, '\n') for x in gene_list]

# Transform attributes into arrays
gene_list = [[occursin(" - ", x) ? split(x, " - ") : SubString{String}[] for x in gene] for gene in gene_list]

# Drop empty entries
gene_list = [gene[.~isempty.(gene)] for gene in gene_list]
println(gene_list[1])

Vector{SubString{String}}[["UNIQUE-ID", "EG11120"], ["TYPES", "BC-8.1"], ["COMMON-NAME", "tfaE"], ["ACCESSION-1", "b1156"], ["ACCESSION-2", "ECK1142"], ["CENTISOME-POSITION", "26.036354    "], ["COMPONENT-OF", "COLI-K12-24"], ["COMPONENT-OF", "TU0-13152"], ["DBLINKS", "(REGULONDB \"b1156\" NIL |kothari| 3713101626 NIL NIL)"], ["DBLINKS", "(STRING \"511145.b1156\" NIL |kothari| 3652459938 NIL NIL)"], ["DBLINKS", "(ECOLIHUB \"tfaE\" NIL |kr| 3474243545 NIL NIL)"], ["DBLINKS", "(ASAP \"ABE-0003880\" NIL |paley| 3398447606 NIL NIL)"], ["DBLINKS", "(ECHOBASE \"EB1110\" NIL |pkarp| 3346767937 NIL NIL)"], ["DBLINKS", "(OU-MICROARRAY \"b1156\" NIL NIL NIL NIL NIL)"], ["IN-GROUP", "PARALOGOUS-GENE-GROUP-154"], ["INSTANCE-NAME-TEMPLATE", "G-*"], ["KNOCKOUT-GROWTH-OBSERVATIONS", "OBS0-439"], ["KNOCKOUT-GROWTH-OBSERVATIONS", "OBS0-49"], ["KNOCKOUT-GROWTH-OBSERVATIONS", "OBS0-37"], ["KNOCKOUT-GROWTH-OBSERVATIONS", "OBS0-33"], ["KNOCKOUT-GROWTH-OBSERVATIONS", "OBS0-44"], ["LAST-UPDATE", "3701017116"

Now that we have the attributes for each gene, we can extract the information we are looking for and store it in a data frame. We iterate through each gene and extract the values for the following attributes: 
- ID,
- Name, 
- Transcription Units, 
- Transcription Direction, 
- Position, 
- Synonyms, 
- Accession ID. 

In [21]:
# Initialize Arrays for storage
ID_list = String[]
name_list = String[]
TU_list = []
direction_list = String[]
position_list = Float64[]
synonym_list = []
accession_list = String[]

# Iterate through every gene
for x in gene_list
    name = filter(x -> x[1] == "UNIQUE-ID", x)
    if ~isempty(name)
        push!(ID_list, name[1][2])
    else
        push!(ID_list, "None")
    end
    
    name = filter(x -> x[1] == "COMMON-NAME", x)
    if ~isempty(name)
        push!(name_list, name[1][2])
    else
        push!(name_list, "None")
    end

    synonyms = filter(x -> x[1] == "SYNONYMS", x)
    _synonym_list = []
    if ~isempty(synonyms)
        for synonym in synonyms
            push!(_synonym_list, synonym[2])
        end
    else
        push!(_synonym_list, "none")
    end
    push!(synonym_list, _synonym_list)
    
    direction = filter(x -> x[1] == "TRANSCRIPTION-DIRECTION", x)
    if ~isempty(direction)
        push!(direction_list, direction[1][2])
    else
        push!(direction_list, "none")
    end

    accession = filter(x -> x[1] == "ACCESSION-1", x)
    if ~isempty(accession)
        push!(accession_list, accession[1][2])
    else
        push!(accession_list, "none")
    end

    components = filter(x -> x[1] == "COMPONENT-OF", x)
    tu_list_gene = []
    if ~isempty(components)
        for component in components
            if occursin("TU", component[2])
                push!(tu_list_gene, component[2])
            end
        end
    else
        push!(tu_list_gene, "none")
    end

    push!(TU_list, tu_list_gene)
    if direction_list[end] == "+"
        position = filter(x -> x[1] == "LEFT-END-POSITION", x)
        if ~isempty(position)
            push!(position_list, parse(Float64, position[1][2]))
        else
            push!(position_list, NaN)
        end  
    elseif direction_list[end] == "-"
        position = filter(x -> x[1] == "RIGHT-END-POSITION", x)
        if ~isempty(position)
            push!(position_list, parse(Float64, position[1][2]))
        else
            push!(position_list, NaN)
        end  
    else 
        push!(position_list, NaN)
    end
end

df_genes = DataFrames.DataFrame(
    ID=ID_list, 
    gene=name_list, 
    direction=direction_list, 
    transcription_units=TU_list, 
    gene_position=position_list, 
    synonyms=synonym_list,
    accession=accession_list
    )

first(df_genes, 5)

,ID,gene,direction,transcription_units,gene_position,synonyms
,String,String,String,Any,Float64,Any
1,EG11120,tfaE,-,"[""TU0-13152""]",1.20912e6,"[""ycfA""]"
2,G0-10603,yrhD,+,[],3.5844e6,"[""none""]"
3,EG11085,rsmH,+,"[""TU0-14439"", ""TU0-941""]",90094.0,"[""yabC"", ""mraW""]"
4,EG12203,cspB,-,"[""TU0-6982""]",1.64155e6,"[""none""]"
5,G0-16721,yabR,-,"[""TU0-44222""]",85511.0,"[""none""]"


Now we can store the DataFrame into a file for easier access later.

In [ ]:
CSV.write("/$home_dir/data/all_genes_table.csv", df_genes)

### Transcription Units

For each gene we have a number of transcription units. A transcription unit is the a set of genes that is transcribed together from a promoter. A gene can be in multiple transcription units either due to multiple promoters or due to differences in transcription termination. Now we need to identify the promoters for each transcription unit.  This information is stored in the file about transcription units. Accessing the information is identical to the way we extracted information about the genes.

In [27]:
# Open Transcription Units
s = open("../data/ecocyc/transunits.dat") do file
    read(file, String)
end

# Drop Comments and split units
tu_list = split(s, "//")[3:end]

# Separate attributes
tu_list = [split(x, '\n') for x in tu_list]

# Transform attributes into arrays
tu_list = [[occursin(" - ", x) ? split(x, " - ") : SubString{String}[] for x in tu] for tu in tu_list]

# Drop empty entries
tu_list = [tu[.~isempty.(tu)] for tu in tu_list]


# Write to DataFrame
ID_list = String[]
promoter_list = []


for x in tu_list
    name = filter(x -> x[1] == "UNIQUE-ID", x)
    if ~isempty(name)
        push!(ID_list, name[1][2])
    else
        push!(ID_list, "None")
    end
    
    components = filter(x -> ((x[1] == "COMPONENTS") && (occursin("PM", x[2]))), x)
    if ~isempty(components)
        for component in components
            if occursin("PM", component[2])
                push!(promoter_list, component[2])
            end
        end
    else
        push!(promoter_list, "none")
    end
end

df_tu = DataFrames.DataFrame(TU_ID=ID_list, promoter_ID=promoter_list)
first(df_tu, 5)

,TU_ID,promoter_ID
,String,Any
1,TU0-12955,none
2,TU0-13608,none
3,TU358,PM439
4,TU0-8519,PM0-9601
5,TU0-8490,PM0-8926


Now that we have the promoters for every transcription unit, we can combine the information we have about genes and transcription units. We sort the data by transcription units, therefore we combine all genes that are part of the same transcription unit.

In [28]:
# Arrays for entry storage
tu_genes = Vector{String}[]
tu_gene_positions = Vector{Float64}[]
tu_direction = String[]

# Iterate through TU
for tu in df_tu.TU_ID
    # Find genes and their information for TU
    _df = df_genes[map(x -> tu in x["transcription_units"], eachrow(df_genes)), ["gene", "direction", "gene_position"]]
    push!(tu_genes, _df.gene)
    push!(tu_gene_positions, _df.gene_position)
    direction = _df.direction |> unique
    # Look for conflicting directions
    if length(direction) > 1
        throw(ErrorException("Found more than one direction for transcription unit $tu."))
    elseif length(direction) == 0
        push!(tu_direction, "")
    else
        push!(tu_direction, direction[1])
    end
end

# Add genes to DataFrame
insertcols!(df_tu, 3, :genes=>tu_genes)
insertcols!(df_tu, 4, :direction=>tu_direction)
insertcols!(df_tu, 5, :gene_position=>tu_gene_positions)

first(df_tu, 5)

,TU_ID,promoter_ID,genes,direction,gene_position
,String,Any,Array…,String,Array…
1,TU0-12955,none,"[""cueR""]",+,[513993.0]
2,TU0-13608,none,"[""yfdE""]",-,[2.48917e6]
3,TU358,PM439,"[""cydA"", ""cydB""]",+,"[771458.0, 773042.0]"
4,TU0-8519,PM0-9601,"[""ryjA""]",-,[4.27807e6]
5,TU0-8490,PM0-8926,"[""pepT""]",+,[1.18584e6]


### Promoters

Now that we have a list of all transcription unites and their genes, we need to extract the information about their promoters. We are looking for the transcription start site of each promoter as well as how much evidence there is for the one that have been identified.

In [31]:
# Read promoter file
s = open("../data/ecocyc/promoters.dat") do file
    read(file, String)
end

# Split promoters and remove comments
promoter_list = split(s, "//")[3:end]

# Separate attributes
promoter_list = [split(x, '\n') for x in promoter_list]

# Transform attributes into arrays
promoter_list = [[occursin(" - ", x) ? split(x, " - ") : SubString{String}[] for x in promoter] for promoter in promoter_list]

# Drop empty entries
promoter_list = [promoter[.~isempty.(promoter)] for promoter in promoter_list]


# Make DataFrame
ID_list = String[]
name_list = String[]
TSS_list = Float64[]
evidence_list = Vector{String}[]

# Iterate through promoters
for x in promoter_list
    name = filter(x -> x[1] == "UNIQUE-ID", x)
    if ~isempty(name)
        push!(ID_list, name[1][2])
    else
        push!(ID_list, "None")
    end
    
    name = filter(x -> x[1] == "COMMON-NAME", x)
    if ~isempty(name)
        push!(name_list, name[1][2])
    else
        push!(name_list, "None")
    end
    
    TSS = filter(x -> x[1] == "ABSOLUTE-PLUS-1-POS", x)
    if ~isempty(TSS)
        push!(TSS_list, parse(Float64, TSS[1][2]))
    else
        push!(TSS_list, NaN)
    end
    
    cits = filter(x -> x[1] == "CITATIONS", x)
    if ~isempty(cits)
        cit_list = String[]
        for cit in cits
            if occursin("EV-EXP", cit[2])
                push!(cit_list, "EXP")
            elseif occursin("EV-COMP", cit[2])
                push!(cit_list, "COMP")
            end
            if length(cit_list) == 0 
                push!(cit_list, "none")
            end
        end
        push!(evidence_list, cit_list)
    else
        push!(evidence_list, String["none"])
    end
    
end
df_tss = DataFrames.DataFrame(promoter_ID=ID_list, promoter=name_list, tss=TSS_list, evidence=evidence_list)
first(df_tss, 5)

,promoter_ID,promoter,tss,evidence
,String,String,Float64,Array…
1,PM00451,uspAp1,3.63998e6,"[""COMP"", ""COMP"", ""EXP"", ""EXP""]"
2,PM0-45328,dedDp3,2.43188e6,"[""COMP""]"
3,PM0-45499,yqeFp11,2.98581e6,"[""COMP""]"
4,PM00628,glnBp3,2.68753e6,"[""EXP""]"
5,PM623,mazEp2,2.91136e6,"[""none"", ""EXP""]"


Now we need to combine the dataframes of transcription units and promoters. After joining, there will be some missing entries when there is a transcription unit that does not have a promoter associated with it. We are going to replace the missing values with easy to interpret entries.

In [34]:
# Join DataFrames
df_joint = outerjoin(df_tu, df_tss, on = :promoter_ID) |> unique

# Replace missing values
df_joint.TU_ID = coalesce.(df_joint.TU_ID, "None")
df_joint.genes = coalesce.(df_joint.genes, [["None"]])
df_joint.direction = coalesce.(df_joint.direction, "0")
df_joint.tss = coalesce.(df_joint.tss, NaN)
df_joint.evidence = coalesce.(df_joint.evidence, [["None"]])
display(first(df_joint, 20))

,TU_ID,promoter_ID,genes,direction
,String,Any,Array…,String
1,TU00431,PM00451,"[""uspA""]",+
2,TU00474,PM00628,"[""glnB""]",-
3,TU523,PM623,"[""mazG"", ""mazE"", ""mazF""]",-
4,TU0-13680,PM0-10103,"[""ffh""]",-
5,TU0-13351,PM0-10409,"[""blr""]",+
6,TU0-14183,PM0-46064,"[""nanX"", ""nanY""]",+
7,TU0-42602,PM0-46356,"[""accA""]",+
8,TU0-36412,PM0-36157,"[""pcnB"", ""folK""]",-
9,TU783,PM924,"[""zraS"", ""zraR""]",+


In [36]:
##
gdf = groupby(df_joint, "promoter_ID")
temp_df = DataFrame()
for _df in gdf
    ind = argmax(maximum(length.(_df.genes)))
    append!(temp_df, _df[ind:ind, :])
end
df_joint = temp_df


# Split DataFrame into promoters with TUs and without
df_joint_prom = df_joint[(df_joint.direction .!= "0") .& (.~ isnan.(df_joint.tss)), :]
CSV.write("../data/promoter_list_ecocyc.csv", df_joint_prom[:, ["promoter", "genes", "gene_position", "direction", "tss", "evidence"]])

"../data/promoter_list_ecocyc.csv"

## RegulonDB

The files from RegulonDB are in `txt` format and can be imported as DataFrames. 

### Genes

First we import the database for genes. We have to do some slight modifications to the dataframe to remove a couple of artifacts.

In [49]:
# Import DataFrame
df_DB_genes = CSV.read(
    "../data/regulonDB/gene.txt", 
    DataFrames.DataFrame, 
    comment="#",
    header=[
        "GENE_ID",
        "GENE_NAME",
        "GENE_POSLEFT",
        "GENE_POSRIGHT",
        "GENE_STRAND",
        "GENE_SEQUENCE",
        "GC_CONTENT",
        "CRI_SCORE",
        "GENE_NOTE",
        "GENE_INTERNAL_COMMENT",
        "KEY_ID_ORG",
        "GENE_TYPE",
    ]
)


# Remove import artifacts
df_DB_genes = df_DB_genes[.~ ismissing.(df_DB_genes.GENE_NAME), :]


# Gene Strand information is missing for some genes, turns out those are split by transposons
df_trans = df_DB_genes[ismissing.(df_DB_genes.GENE_STRAND), :]
df_DB_genes = df_DB_genes[.~ ismissing.(df_DB_genes.GENE_STRAND), :]

# Record position of gene in genome
gene_pos = []
for i in 1:nrow(df_DB_genes)
    if df_DB_genes[i, "GENE_STRAND"] == "forward"
        push!(gene_pos, df_DB_genes[i, "GENE_POSLEFT"])
    else
        push!(gene_pos, df_DB_genes[i, "GENE_POSRIGHT"])
    end
end

# Add gene location to dataframe
insertcols!(df_DB_genes, 13, :gene_position => gene_pos)

# Print first 5 rows
first(df_DB_genes, 5)

,GENE_ID,GENE_NAME,GENE_POSLEFT,GENE_POSRIGHT,GENE_STRAND,GENE_SEQUENCE
,String15,String31?,Int64?,Int64?,String7?,String?
1,ECK120000001,alr,4265782,4266861,forward,ATGCAAGCGGCAACTGTTGTGATTAACCGCCGCGCTCTGCGACACAACCTGCAACGTCTTCGTGAACTGGCCCCTGCCAGTAAAATGGTTGCGGTGGTGAAAGCGAACGCTTATGGTCACGGTCTTCTTGAGACCGCGCGAACGCTCCCCGATGCTGACGCCTTTGGCGTAGCCCGTCTCGAAGAAGCTCTGCGACTGCGTGCGGGGGGAATCACCAAACCTGTACTGTTACTCGAAGGCTTTTTTGATGCCAGAGATCTGCCGACGATTTCTGCGCAACATTTTCATACCGCCGTGCATAACGAAGAACAGCTGGCTGCGCTGGAAGAGGCTAGCCTGGACGAGCCGGTTACCGTCTGGATGAAACTCGATACCGGTATGCACCGTCTGGGCGTAAGGCCGGAACAGGCTGAGGCGTTTTATCATCGCCTGACCCAGTGCAAAAACGTTCGTCAGCCGGTGAATATCGTCAGCCATTTTGCGCGCGCGGATGAACCAAAATGTGGCGCAACCGAGAAACAACTCGCTATCTTTAATACCTTTTGCGAAGGCAAACCTGGTCAACGTTCCATTGCCGCGTCGGGTGGCATTCTGCTGTGGCCACAGTCGCATTTTGACTGGGTGCGCCCGGGCATCATTCTTTATGGCGTCTCGCCGCTGGAAGATCGCTCCACCGGTGCCGATTTTGGCTGTCAGCCAGTGATGTCACTAACCTCCAGCCTGATTGCCGTGCGTGAGCATAAAGCCGGAGAGCCTGTTGGTTATGGTGGAACCTGGGTAAGCGAACGTGATACCCGTCTTGGCGTAGTCGCGATGGGCTATGGCGATGGTTATCCGCGCGCCGCGCCGTCCGGTACGCCAGTGCTGGTGAACGGTCGCGAAGTACCGATTGTCGGGCGCGTGGCGATGGATATGATCTGCGTAGACTTAGGTCCACAGGCGCAGGACAAAGCCGGGGATCCGGTCATTTTATGGGGCGAAGGTTTGCCCGTAGAACGTATCGCTGAAATGACGAAAGTAAGCGCTTACGAACTTATTACGCGCCTGACTTCAAGGGTCGCGATGAAATACGTGGATTAA
2,ECK120000002,modB,795862,796551,forward,ATGATACTGACCGATCCAGAATGGCAGGCAGTTTTATTAAGCCTGAAAGTTTCTTCCCTGGCTGTGCTGTTTAGCCTGCCGTTTGGGATCTTTTTTGCCTGGTTACTGGTGCGTTGCACGTTTCCGGGCAAAGCTCTGCTCGACAGCGTACTGCATCTACCGCTGGTGTTACCGCCCGTGGTCGTCGGTTACTTATTATTAGTTTCGATGGGACGGCGCGGATTTATCGGTGAACGTCTGTATGACTGGTTTGGTATTACCTTCGCCTTTAGCTGGCGCGGCGCGGTTCTCGCTGCCGCCGTCATGTCGTTTCCGCTGATGGTGCGGGCAATTCGTCTGGCGCTGGAAGGGGTTGATGTCAAACTGGAACAGGCCGCAAGAACACTGGGGGCCGGGCGCTGGCGCGTTTTCTTTACTATCACGTTACCGCTGACCTTACCGGGAATTATTGTTGGTACGGTACTGGCTTTTGCTCGTTCTCTCGGTGAGTTTGGTGCAACCATCACCTTTGTGTCGAACATTCCTGGTGAAACGCGAACCATTCCTTCTGCCATGTATACCCTGATCCAGACCCCCGGCGGGGAAAGTGGAGCGGCGAGACTGTGCATTATTTCTATTGCGCTGGCGATGATCTCCCTGTTGATTTCAGAATGGCTGGCCAGAATCAGCCGTGAACGGGCGGGGCGCTAA
3,ECK120000003,cysZ,2531463,2532224,forward,ATGGTTTCATCATTCACATCTGCCCCACGCAGCGGTTTTTACTATTTTGCGCAAGGCTGGAAGCTCGTCTCGCAACCTGGGATTCGGCGTTTCGTTATTTTACCGCTGCTGGTCAATATTTTGTTGATGGGGGGCGCATTCTGGTGGCTCTTTACACAGCTCGATGTCTGGATCCCGACTCTCATGAGTTACGTTCCGGACTGGCTACAATGGCTGAGTTATCTGTTGTGGCCTCTGGCTGTCATCTCTGTGCTGTTAGTGTTTGGCTATTTCTTCTCCACGATTGCTAACTGGATTGCCGCTCCGTTTAACGGTTTATTGGCTGAACAACTGGAAGCACGATTGACTGGCGCTACACCGCCAGATACCGGGATTTTCGGTATCATGAAAGATGTGCCGCGAATCATGAAACGCGAATGGCAAAAATTTGCCTGGTATCTGCCGCGCGCAATTGTATTACTAATTCTTTACTTCATCCCTGGTATTGGGCAAACCGTCGCGCCGGTACTGTGGTTCCTGTTTAGCGCCTGGATGTTAGCCATCCAGTATTGCGATTACCCCTTCGATAACCACAAAGTGCCGTTTAAAGAGATGCGCACCGCCCTGCGCACACGCAAAATCACCAATATGCAGTTTGGTGCTTTAACCAGCCTGTTTACGATGATCCCGCTGCTTAATCTGTTCATCATGCCCGTTGCCGTTTGTGGCGCGACGGCGATGTGGGTCGATTGCTATCGCGATAAACACGCGATGTGGCGGTAA
4,ECK120000004,dfp,3812731,3813951,forward,ATGAGCCTGGCCGGTAAAAAAATCGTTCTCGGCGTTAGCGGCGGTATTGCTGCCTATAAAACCCCTGAACTGGTGCGTCGTTTGCGCGATCGCGGGGCCGACGTCCGCGTAGCCATGACCGAAGCGGCAAAAGCCTTTATCACCCCACTTAGCTTGCAGGCGGTTTCTGGTTATCCCGTTTCCGACAGTCTGCTGGACCCGGCAGCCGAAGCCGCTATGGGCCATATTGAGCTGGGTAAATGGGCTGATTTAGTGATTCTCGCCCCTGCCACGGCAGATTTGATTGCCCGTGTTGCTGCCGGAATGGCGAATGACCTGGTATCGACGATTTGTCTGGCTACACCTGCGCCTGTAGCCGTGCTCCCCGCCATGAACCAGCAGATGTACCGTGCCGCTGCCACGCAGCATAATTTAGAGGTGCTTGCTTCCCGTGGTTTGCTCATCTGGGGGCCAGACAGTGGCAGTCAGGCTTGTGGTGATATCGGTCCTGGGCGAATGCTCGATCCGTTAACCATTGTGGATATGGCGGTAGCGCATTTTTCGCCCGTCAACGACCTGAAACATCTGAACATTATGATTACCGCCGGCCCGACGCGTGAACCGCTCGATCCGGTGCGTTATATCTCTAATCACAGCTCCGGCAAGATGGGTTTTGCTATCGCCGCCGCCGCTGCCCGTCGTGGCGCGAACGTCACGCTGGTATCAGGTCCGGTTTCACTACCGACGCCACCGTTTGTTAAACGTGTTGATGTGATGACCGCGCTGGAAATGGAAGCCGCCGTGAATGCTTCTGTACAGCAGCAAAATATTTTTATCGGCTGCGCCGCCGTGGCGGATTATCGCGCAGCTACCGTGGCCCCAGAGAAAATCAAAAAGCAGGCCACGCAGGGTGATGAATTAACAATAAAAATGGTTAAAAACCCCGATATCGTCGCAGGCGTTGCCGCACTAAAAGACCATCGACCCTACGTCGTTGGATTTGCCGCCGAAACAAATAATGTGGAAGAATACGCCCGGCAAAAACGTATCCGTAAAAACCTTGATCTGATCTGCGCGAACGATGTTTCCCAGCCAACTCAAGGATTTAACAGCGACAACAACGCATTACACCTTTTCTGGCAGGACGGAGATAAAGTCTTACCGCTTGAGCGCAAAGAGCTCCTTGGCCAA

### Transcription Units

Same procedure for the transcription units.

In [52]:
df_DB_TU = CSV.read(
    "../data/regulonDB/transcription_unit.txt", 
    DataFrames.DataFrame, 
    comment="#",
    header=[
        "TRANSCRIPTION_UNIT_ID",
        "PROMOTER_ID",
        "TRANSCRIPTION_UNIT_NAME",
        "OPERON_ID",
        "KEY_ID_ORG",
        "TRANSCRIPTION_UNIT_NOTE",
        "TU_INTERNAL_COMMENT",
    ],
    delim='\t'
)

# Remove import artifacts
df_DB_TU = df_DB_TU[.~ ismissing.(df_DB_TU.TRANSCRIPTION_UNIT_ID), :]

# Remove transcription units without promoters
df_DB_TU = df_DB_TU[.~ ismissing.(df_DB_TU.PROMOTER_ID), :]
df_DB_TU.PROMOTER_ID = coalesce.(df_DB_TU.PROMOTER_ID, "None")

# Fix type
df_DB_TU.TRANSCRIPTION_UNIT_ID = convert(Vector{String}, df_DB_TU.TRANSCRIPTION_UNIT_ID)
df_DB_TU.PROMOTER_ID = convert(Vector{String}, df_DB_TU.PROMOTER_ID)

# Print first 5 columns
df_DB_TU

,TRANSCRIPTION_UNIT_ID,PROMOTER_ID,TRANSCRIPTION_UNIT_NAME,OPERON_ID
,String,String,String?,String15?
1,ECK120008913,ECK120009851,astCADBE,ECK120014360
2,ECK120008914,ECK120009852,astCADBE,ECK120014360
3,ECK120008915,ECK120009853,astCADBE,ECK120014360
4,ECK120008916,ECK120009855,nrdHIEF,ECK120014362
5,ECK120008917,ECK120010879,cpxPQ,ECK125242769
6,ECK120008918,ECK120009856,srkA-dsbA,ECK125272742
7,ECK120008919,ECK120009857,dsbA,ECK125272742
8,ECK120008921,ECK120009858,dmsABC,ECK120014367
9,ECK120008922,ECK120009860,moaABCDE,ECK120014368


There is a file that connects the transcription units with genes, so we import it to combine the dataframes later.

In [54]:

df_DB_link = CSV.read(
    "../data/regulonDB/tu_gene_link.txt", 
    DataFrames.DataFrame, 
    comment="#",
    header=[
        "TRANSCRIPTION_UNIT_ID",
        "GENE_ID",
    ],
    delim='\t'
)
first(df_DB_link, 5)

,TRANSCRIPTION_UNIT_ID,GENE_ID
,String15,String15
1,ECK120008913,ECK120003528
2,ECK120008913,ECK120003529
3,ECK120008913,ECK120003530
4,ECK120008913,ECK120003531
5,ECK120008913,ECK120003532


### Promoters

Lastly, we import the promoters.

In [46]:
# Import promoter data
df_DB_prom = CSV.read(
    "../data/regulonDB/promoter.txt", 
    DataFrames.DataFrame, 
    comment="#", 
    header=[
        "PROMOTER_ID",
        "PROMOTER_NAME",
        "PROMOTER_STRAND",
        "POS_1",
        "SIGMA_FACTOR",
        "BASAL_TRANS_VAL",
        "EQUILIBRIUM_CONST",
        "KINETIC_CONST",
        "STRENGTH_SEQ",
        "PROMOTER_SEQUENCE",
        "KEY_ID_ORG",
        "PROMOTER_NOTE",
        "PROMOTER_INTERNAL_COMMENT",
    ]
)

# Remove import artifacts
df_DB_prom = df_DB_prom[.~ ismissing.(df_DB_prom.PROMOTER_NAME), :]

# Change direction to be compatible with other database
direction_dict = Dict("forward" => "+", "reverse" => "-")
df_DB_prom.PROMOTER_STRAND = [direction_dict[x] for x in df_DB_prom.PROMOTER_STRAND]

# Fix type
df_DB_prom.PROMOTER_NAME = convert(Vector{String}, df_DB_prom.PROMOTER_NAME)

# Show first five rows
first(df_DB_prom, 5)

┌ Warning: thread = 1 warning: only found 12 / 13 columns around data row: 1006. Filling remaining columns with `missing`
└ @ CSV /Users/tomroschinger/.julia/packages/CSV/0Elut/src/file.jl:577
┌ Warning: thread = 1 warning: only found 12 / 13 columns around data row: 1006. Filling remaining columns with `missing`
└ @ CSV /Users/tomroschinger/.julia/packages/CSV/0Elut/src/file.jl:577
┌ Warning: thread = 1 warning: only found 1 / 13 columns around data row: 1007. Filling remaining columns with `missing`
└ @ CSV /Users/tomroschinger/.julia/packages/CSV/0Elut/src/file.jl:577
┌ Warning: thread = 1 warning: only found 1 / 13 columns around data row: 1007. Filling remaining columns with `missing`
└ @ CSV /Users/tomroschinger/.julia/packages/CSV/0Elut/src/file.jl:577
┌ Warning: thread = 1 warning: only found 2 / 13 columns around data row: 1008. Filling remaining columns with `missing`
└ @ CSV /Users/tomroschinger/.julia/packages/CSV/0Elut/src/file.jl:577


,PROMOTER_ID,PROMOTER_NAME,PROMOTER_STRAND,POS_1,SIGMA_FACTOR,BASAL_TRANS_VAL
,String,String,String,Int64?,String7?,Missing
1,ECK120009842,galRp,+,2976569,Sigma70,missing
2,ECK120009843,lpxLp,-,1116709,missing,missing
3,ECK120009844,yceAp,+,1116772,missing,missing
4,ECK120009845,mraZp,+,89596,Sigma70,missing
5,ECK120009846,sohBp1,+,1329284,Sigma70,missing


Now we combine genes, transcription units and promoters. We go through each transcription unit, look for the associated genes and identify the corresponding promoter. Then, the information is combined and stored in a new dataframe.

In [57]:
df_DB_tss = DataFrames.DataFrame()

# Iterate through transcription units
for i in 1:nrow(df_DB_TU)
    # Get TU ID and promoter ID
    TU_ID, promoter_ID = df_DB_TU[i, ["TRANSCRIPTION_UNIT_ID", "PROMOTER_ID"]]
    
    # Find gene IDs of genes in TU
    gene_IDs = df_DB_link[map(x -> x == TU_ID, df_DB_link.TRANSCRIPTION_UNIT_ID), "GENE_ID"]

    # Get gene names and locations
    genes = convert(Vector{String}, df_DB_genes[map(x -> x in gene_IDs, df_DB_genes.GENE_ID), "GENE_NAME"])
    gene_positions = convert(Vector{Int64}, df_DB_genes[map(x -> x in gene_IDs, df_DB_genes.GENE_ID), "gene_position"])

    # Get promoter data and create temporary dataframe
    promoter_df = df_DB_prom[df_DB_prom.PROMOTER_ID .== promoter_ID, ["PROMOTER_NAME", "PROMOTER_STRAND", "POS_1",]]
    
    # Insert data to temporary dataframe
    insertcols!(promoter_df, 2, :genes =>fill(genes, nrow(promoter_df)))
    insertcols!(promoter_df, 3, :gene_position =>fill(gene_positions, nrow(promoter_df)))
    
    # Rename columns
    rename!(
        promoter_df,
        Dict(
            "PROMOTER_NAME"=>"promoter",
            "PROMOTER_STRAND"=>"direction",
            "POS_1"=>"tss"
        )
    )
    
    # Add temporary dataframe to target
    append!(df_DB_tss, promoter_df)
end

first(df_DB_tss, 5)

,promoter,genes,gene_position
,String,Array…,Array…
1,astCp1,"[""astE"", ""astB"", ""astD"", ""astA"", ""astC""]","[1826923, 1828259, 1829734, 1830765, 1831982]"
2,astCp2,"[""astE"", ""astB"", ""astD"", ""astA"", ""astC""]","[1826923, 1828259, 1829734, 1830765, 1831982]"
3,astCp3,"[""astE"", ""astB"", ""astD"", ""astA"", ""astC""]","[1826923, 1828259, 1829734, 1830765, 1831982]"
4,nrdHp,"[""nrdF"", ""nrdE"", ""nrdH"", ""nrdI""]","[2803502, 2801348, 2800723, 2800965]"
5,cpxPp,"[""cpxP"", ""cpxQ""]","[4105820, 4106330]"


In [60]:
CSV.write("../data/promoter_list_regulon_DB.csv", df_DB_tss)

"../data/promoter_list_regulon_DB.csv"

### Evidence

In [74]:
# Import DataFrame
df_evidence = CSV.read(
    "../data/regulonDB/evidence.txt", 
    DataFrames.DataFrame, 
    comment="#",
    header=[
            "EVIDENCE_ID",
            "EVIDENCE_NAME",
            "TYPE_OBJECT",
            "EVIDENCE_CODE",
            "EVIDENCE_NOTE",
            "EVIDENCE_INTERNAL_COMMENT",
            "KEY_ID_ORG",
            "EVIDENCE_TYPE",
            "EVIDENCE_CATEGORY",
            "HEAD",
            "EXAMPLE",
            "ECOCYC_CODE",
            "EVIDENCE_TECH_CODE"
    ]
)

df_evidence.EVIDENCE_ID

102-element Vector{String15}:
 "GID000000001"
 "GID000000002"
 "GID000000003"
 "GID000000004"
 "GID000000005"
 "GID000000006"
 "GID000000007"
 "GID000000008"
 "GID000000009"
 "GID000000010"
 "GID000000011"
 "GID000000012"
 "GID000000013"
 ⋮
 "GID000064260"
 "GID000064261"
 "GID000064262"
 "GID000064263"
 "GID000064264"
 "GID000064265"
 "GID000064266"
 "GID000064267"
 "GID000064268"
 "GID000064269"
 "GID000064270"
 "GID0065864  "

## Combine Sources

In [61]:
# Promoters from Ecocyc
promoter_list_ecocyc = CSV.read(
    "../data/promoter_list_ecocyc.csv", 
    DataFrames.DataFrame, 
    types=Dict(
        "promoter"=>String,
        "tss"=>Float64,
        "direction"=>String
    )
)

# Promoters from RegulonDB
promoter_list_regulonDB = CSV.read(
    "../data/promoter_list_regulon_DB.csv", 
    DataFrames.DataFrame, 
    types=Dict(
        "promoter"=>String,
        "tss"=>Float64,
        "direction"=>String
    )
)

,promoter,genes,gene_position
,String,String,String
1,astCp1,"[""astE"", ""astB"", ""astD"", ""astA"", ""astC""]","[1826923, 1828259, 1829734, 1830765, 1831982]"
2,astCp2,"[""astE"", ""astB"", ""astD"", ""astA"", ""astC""]","[1826923, 1828259, 1829734, 1830765, 1831982]"
3,astCp3,"[""astE"", ""astB"", ""astD"", ""astA"", ""astC""]","[1826923, 1828259, 1829734, 1830765, 1831982]"
4,nrdHp,"[""nrdF"", ""nrdE"", ""nrdH"", ""nrdI""]","[2803502, 2801348, 2800723, 2800965]"
5,cpxPp,"[""cpxP"", ""cpxQ""]","[4105820, 4106330]"
6,srkAp,"[""dsbA"", ""srkA""]","[4043418, 4042415]"
7,dsbAp,"[""dsbA""]",[4043418]
8,dmsAp2,"[""dmsA"", ""dmsB"", ""dmsC""]","[940959, 943414, 944033]"
9,moaAp2,"[""moaA"", ""moaB"", ""moaD"", ""moaE"", ""moaC""]","[817044, 818055, 819048, 819295, 818570]"


In [62]:
# Replace columns by nicer types
promoter_list_ecocyc.genes = parse.(Vector{String}, promoter_list_ecocyc.genes)
promoter_list_ecocyc.evidence = parse.(Vector{String}, promoter_list_ecocyc.evidence)
promoter_list_ecocyc.gene_position = parse.(Vector{Float64}, promoter_list_ecocyc.gene_position)
promoter_list_regulonDB.genes = parse.(Vector{String}, promoter_list_regulonDB.genes)
promoter_list_regulonDB.gene_position = parse.(Vector{Float64}, promoter_list_regulonDB.gene_position)

2357-element Vector{Vector{Float64}}:
 [1.826923e6, 1.828259e6, 1.829734e6, 1.830765e6, 1.831982e6]
 [1.826923e6, 1.828259e6, 1.829734e6, 1.830765e6, 1.831982e6]
 [1.826923e6, 1.828259e6, 1.829734e6, 1.830765e6, 1.831982e6]
 [2.803502e6, 2.801348e6, 2.800723e6, 2.800965e6]
 [4.10582e6, 4.10633e6]
 [4.043418e6, 4.042415e6]
 [4.043418e6]
 [940959.0, 943414.0, 944033.0]
 [817044.0, 818055.0, 819048.0, 819295.0, 818570.0]
 [1.157869e6]
 [1.157869e6]
 [2.716449e6]
 [209679.0]
 ⋮
 [2.545609e6]
 [2.804815e6, 2.80601e6, 2.807132e6]
 [2.535834e6, 2.533764e6, 2.534066e6]
 [792055.0, 789979.0, 791029.0, 788837.0]
 [4.414275e6]
 [3.600783e6]
 [3.056949e6]
 [4.111507e6]
 [1.937649e6]
 [3.882321e6, 3.881221e6]
 [1.032139e6, 1.033254e6, 1.035066e6, 1.03577e6, 1.036354e6, 1.036749e6]
 [2.313084e6]

In [63]:
# Join the datasets
df = promoter_list_ecocyc[(.~ isnan.(promoter_list_ecocyc.tss)) .& (map(x-> x != ["None"], promoter_list_ecocyc.genes)), :]
promoter_list_regulonDB.tss = coalesce.(promoter_list_regulonDB.tss, NaN)
append!(df, promoter_list_regulonDB)

LoadError: ArgumentError: Column names :evidence were found in only one of the passed data frames and `cols == :setequal`

# Computational Environment

In [42]:
using Pkg
Pkg.status(["DataFrames", "CSV"])

     Project wgregseq v0.1.0
      Status `~/git/1000_genes_ecoli/Project.toml`
  [336ed68f] CSV v0.10.2
  [a93c6f00] DataFrames v1.3.2
